Faster and using half of parameter FastText Model

In [2]:
def generate_bigrams(x):
  n_grams = set(zip(*[x[i:] for i in range(2)]))
  for n_gram in n_grams:
    x.append(' '.join(n_gram))
  return x

In [3]:
#check as test
generate_bigrams(['This', 'film', 'is', 'good'])

['This', 'film', 'is', 'good', 'film is', 'is good', 'This film']

In [4]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

SEED  =1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministics = True

TEXT  = data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm',
                   preprocessing = generate_bigrams)

LABEL = data.LabelField(dtype = torch.float)

In [5]:
#loading IMDB dataset
import random

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, val_data = train_data.split(random_state = random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 344k/84.1M [00:00<00:26, 3.11MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:00<00:00, 84.4MB/s]


In [6]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.39MB/s]                           
100%|█████████▉| 398653/400000 [00:14<00:00, 28315.44it/s]

In [22]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [8]:
#Model has few parameter and 2 layer with no RNN component 
# Has embedding layer and linear layer

In [23]:
# import torch.nn as nn
# import torch.nn.functional as F

# class FastText(nn.Module):

#   def __init__(self, vocab_size, emb_dim, o_dim, pad_idx):
#     super().__init__()

#     self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx = pad_idx)
#     self.fc = nn.Linear(emb_dim, o_dim)

#   def forward(self, text):

#     embedded = self.embedding(text)
#     embedded = self.embedded.permute(1,0,2)
#     pooled = F.avg_pool2d(embedded, (embedded.shape[1],1)).squeeze(1)

#     return self.fc(pooled)
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

In [24]:
# I_dim = len(TEXT.vocab)
# Emb_dim = 100
# Out_dim = 1
# Pad_Idx = TEXT.vocab.stoi[TEXT.pad_token]

# model  = FastText(I_dim, Emb_dim, Out_dim, Pad_Idx)
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [25]:
# def count_parameters(model):
#   return sum(p.numel() for p in model.parameters() if p.requires_grad)

# print(f"The Model has {count_parameters(model):,} trainable parameters")
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [26]:
# pretrained_embs = TEXT.vocab.vectors

# model.embedding.weight.data.copy_(pretrained_embs)
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.0530, -1.0757,  0.3903,  ...,  0.0792, -0.3059,  1.9734],
        [ 0.5571, -0.3352,  0.0417,  ...,  0.0257,  1.1868, -0.8791],
        [ 0.8085,  0.2957, -0.9159,  ...,  0.3877, -0.8750,  1.6714]])

In [27]:
# Unk_Idx = TEXT.vocab.stoi[TEXT.unk_token]
# #print(Unk_Idx)
# model.embedding.weight.data[Unk_Idx] = torch.zeros(Emb_dim)
# model.embedding.weight.data[Pad_Idx] = torch.zeros(Emb_dim)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [28]:
# #Training the model 
# import torch.optim as optim

# optimizer = optim.Adam(model.parameters())
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [29]:
# criterion = nn.BCEWithLogitsLoss()
# model = model.to(device)
# criterion = criterion.to(device)
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [30]:
# #Implementation of function for accuracy
# def bin_acc(preds, y):

#   rounded_preds = torch.round(torch.sigmoid(preds))
#   correct = (rounded_preds == y).float()
#   acc = correct.sum() / len(correct)
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [35]:
# def train(model, iterator, optimizor, criterion):

#   epoch_loss = 0
#   epoch_acc = 0

#   model.train()

#   for batch in iterator:

#     optimizer.zero_grad()
#     predictions = model(batch.text).squeeze(1)
#     loss = criterion(predictions, batch.label)
#     acc = bin_acc(predictions, batch.label)

#     loss.backward()
#     optimizer.step()

#     epoch_loss += loss.item()
#     epoch_acc += acc.item()

#   return epoch_loss / len(iterator), epoch_acc / len(iterator)



def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [36]:
# def evaluate(model, iterator, criterion):

#   epoch_loss = 0
#   epoch_acc = 0

#   model.eval()

#   with torch.no_grad():

#     for batch in iterator:

#       predictions = model(batch.text).squeeze(1)
#       loss = criterion(predictions, batch.label)
#       acc = bin_acc(predictions, batch.label)

#       epoch_loss += loss.item()
#       #epoch_acc += acc.item()

#   return epoch_loss/len(iterator), epoch_acc/len(iterator)
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
# import time

# def epoch_times(s_time, e_time):
#   elapsed_time = e_time - s_time
#   elapsed_min = int(elapsed_time / 60)
#   elapsed_sec = int(elapsed_time - (elapsed_min * 60))
#   return elapsed_min, elapsed_sec
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
Num_Epochs = 5

# best_val_loss = float("inf")

# for epoch in range(Num_Epochs):

#   s_time = time.time()

#   train_loss, train_acc = train(model, train_itr, optimizer, criterion)
#   val_loss, val_acc = evaluate(model, valid_itr, criterion)

#   e_time = time.time()

#   epoch_min, epoch_sec = epoch_times(s_time, e_time)

#   if val_loss < best_val_loss:
#     best_val_loss = val_loss
#     torch.save(model.state_dict(), 'fasttext-model.pt')

#     print(f'Epoch: {epoch+1: 02} | Epoch Time: {epoch_min}m {epoch_sec}s')
#     print(f'\tTrain Loss: {train_loss:.3f} | Train Accuracy: {train_acc*100:.2f}%')
#     print(f'\tValidation Loss: {val_loss:.3f} | Validation Accuracy: {val_acc*100:.2f}%')
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 0.686 | Train Acc: 58.24%
	 Val. Loss: 0.630 |  Val. Acc: 69.38%
Epoch: 02 | Epoch Time: 0m 9s
	Train Loss: 0.645 | Train Acc: 71.93%
	 Val. Loss: 0.515 |  Val. Acc: 74.06%
Epoch: 03 | Epoch Time: 0m 9s
	Train Loss: 0.570 | Train Acc: 79.66%
	 Val. Loss: 0.432 |  Val. Acc: 80.73%
Epoch: 04 | Epoch Time: 0m 9s
	Train Loss: 0.494 | Train Acc: 84.30%
	 Val. Loss: 0.401 |  Val. Acc: 83.70%
Epoch: 05 | Epoch Time: 0m 9s
	Train Loss: 0.430 | Train Acc: 87.05%
	 Val. Loss: 0.394 |  Val. Acc: 85.76%


In [39]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.389 | Test Acc: 85.40%


In [40]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [43]:
predict_sentiment(model, "This film is terrible")

5.779239700132166e-07

In [42]:
predict_sentiment(model, "This film is great")

1.0